In [5]:
using Pkg
Pkg.activate(".")
using Vizagrams
using DataFrames
using Random
using Distributions
using Colors
using StructArrays
using LinearAlgebra

  Activating project at `~/Documents/GitHub/Vizagrams.jl/tutorials`


In [6]:
struct SmoothWire <: Mark
    ptstart
    ptend
    hstart
    hend
end
SmoothWire(;ptstart=[0,0.5],ptend=[1,1.5],hstart=1,hend=1) = SmoothWire(ptstart,ptend,hstart,hend)
function Vizagrams.ζ(w::SmoothWire)::TMark
    (;ptstart, ptend, hstart, hend) = w
    stop = ptstart + [0,hstart/2]
    sbot = ptstart + [0,-hstart/2]
    etop = ptend + [0,hend/2]
    ebot = ptend + [0,-hend/2]
    midx = (ptend[1] + ptstart[1])/2
    q = CBezierPolygon(
        [sbot,stop,etop,ebot],
        [sbot,stop,[midx, stop[2]],[midx,etop[2]],etop,ebot,[midx,ebot[2]],[midx,sbot[2]]]
    )
    return dmlift(q)
end

In [7]:
draw(SmoothWire())

<svg height="300" viewBox="-0.03333333333333333 -2.033333333333333 1.0666666666666667 2.066666666666667" width="150.0" preserveAspectRatio="xMidYMid meet" xmlns="http://www.w3.org/2000/svg" version="1.1"><path style="vector-effect: non-scaling-stroke; stroke: black; fill: none; " transform="scale(1,-1)" d="M 0.0 0.0 C 0.0 0.0 0.0 1.0 0.0 1.0 0.5 1.0 0.5 2.0 1.0 2.0 1.0 2.0 1.0 1.0 1.0 1.0 C 0.5 1.0 0.5 0.0 0.0 0.0  Z" /></svg>

In [8]:

# Sample from the distribution
# random_value = countries[rand(dist)]
# Random.seed!(4)
function random_student()
    countries = ["CHINA","USA","AUSTRALIA","INDIA","OTHER","NOTHING"]
    grad = ["UNDERGRADUATE","GRADUATE","POST-GRADUATE","NOTHING"]
    w = [5, 3, 1,1,1]  # Define your weights here
    w = w/sum(w)
    
    # Create a weighted distribution
    dist = Distributions.Categorical(w)
    under = countries[rand(dist)]
    index = findfirst(x -> x == under, countries)
    w = [4, 20, 5,9,2,5]  # Define your weights here
    w = w/sum(w)
    w[index] = 1.0
    w = w/sum(w)
    dist = Distributions.Categorical(w)
    grad = countries[rand(dist)]
    if grad == "NOTHING"
        return under, grad, "NOTHING"
    end
    index = findfirst(x -> x == grad, countries)
    w = [4, 8, 5,9,2,10]  # Define your weights here
    w = w/sum(w)
    distgrad = Distributions.Categorical(w)
    postgrad = countries[rand(dist)]
    return under, grad, postgrad
end

Random.seed!(4)
df = DataFrame(map(x->random_student(),1:1000));
df = rename(df,[:under,:grad,:pos]);

df[!,:count] = map(x->1, eachrow(df));

In [9]:
gdf = combine(groupby(df,[:under,:grad]),:count=>sum)
gdf[!,:vertex] = map(row->"under_"*row[:under]*"_" * row[:grad], eachrow(gdf))
gdf = rename(gdf,:under=>:country,:count_sum=>:w)
gdf[!,:degree] = map(row->"under",eachrow(gdf))

vert = copy(gdf[!,[:vertex,:country,:degree,:w]])

gdf = combine(groupby(df,[:grad,:pos]),:count=>sum)
gdf[!,:vertex] = map(row->"grad_"*row[:grad]*"_" * row[:pos], eachrow(gdf))
gdf = rename(gdf,:grad=>:country,:count_sum=>:w)
gdf[!,:degree] = map(row->"grad",eachrow(gdf))
gdf = gdf[gdf[!,:country] .!= "NOTHING",:]

vert = vcat(vert,copy(gdf[!,[:vertex,:country,:degree,:w]]))

gdf = gdf[gdf[!,:pos] .!= "NOTHING",:]
gdf[!,:degree] = map(row->"pos",eachrow(gdf))
gdf = rename(gdf,:country=>:grad)
gdf[!,:vertex] = map(row->"pos_"*row[:grad]*"_" * row[:pos], eachrow(gdf))
gdf = rename(gdf,:pos=>:country)

vert = vcat(vert,copy(gdf[!,[:vertex,:country,:degree,:w]]))

vert = sort(vert,:w,rev=true);

In [10]:
dv = combine(groupby(vert,[:country,:degree]),:w=>sum,renamecols=false)

dv[!,:v] = map(row->row[:degree]*"_"*row[:country],eachrow(dv));

In [11]:
plt = Plot(
    title="MyPlot",
    figsize=(800,340)./2,
    data=dv,
    encodings=(
        x=(field=:degree,datatype=:n,guide=(tickvalues=["under","grad","pos"],)),
        y=(field=:w,datatype=:q,guide=(lim = (0,1500),)),
        color=(field=:country,datatype=:n),
        country=(field=:country,datatype=:n,scale=IdScale()),
        degree=(field=:degree,datatype=:n,scale=IdScale()),
    ),
    graphic =
            ∑(i=:x,op=+,
                ∑(i=:color,op=op=(x,y)->x↑(T(0,10),y),orderby=:country, descend=true,
                    ∑(row -> begin
                        S(:fill=>row[:color],:id=>row[:degree]*"_"*row[:country])*
                    Bar(h=row[:y],c=[row[:x],0], w =35)
                    end
                ))
        )
)
draw(plt,height=500)

<svg height="500" viewBox="-44.81098307291668 -201.53949869791666 517.7918880208333 241.2319921875" width="1084.6883972394958" preserveAspectRatio="xMidYMid meet" xmlns="http://www.w3.org/2000/svg" version="1.1"><polygon points="0.0 0.0 400.0 0.0 400.0 170.0 0.0 170.0 " style="vector-effect: non-scaling-stroke; stroke: lightgrey; fill: white; fill-opacity: 0; " transform="scale(1,-1)" /><polyline points="80.0 0.0 80.00000000000001 170.0 " style="vector-effect: non-scaling-stroke; stroke: grey; fill: none; opacity: 0.6; stroke-dasharray: 2; " transform="scale(1,-1)" /><polyline points="200.0 0.0 200.0 170.0 " style="vector-effect: non-scaling-stroke; stroke: grey; fill: none; opacity: 0.6; stroke-dasharray: 2; " transform="scale(1,-1)" /><polyline points="320.0 0.0 320.0 170.0 " style="vector-effect: non-scaling-stroke; stroke: grey; fill: none; opacity: 0.6; stroke-dasharray: 2; " transform="scale(1,-1)" /><polyline points="0.0 0.0 400.0 0.0 " style="vector-effect: non-scaling-stroke; stroke: grey; fill: none; opacity: 0.6; stroke-dasharray: 2; " transform="scale(1,-1)" /><polyline points="3.4698325975841673e-15 56.666666666666664 400.0 56.666666666666664 " style="vector-effect: non-scaling-stroke; stroke: grey; fill: none; opacity: 0.6; stroke-dasharray: 2; " transform="scale(1,-1)" /><polyline points="6.9396651951683345e-15 113.33333333333333 400.0 113.33333333333333 " style="vector-effect: non-scaling-stroke; stroke: grey; fill: none; opacity: 0.6; stroke-dasharray: 2; " transform="scale(1,-1)" /><polyline points="1.0409497792752502e-14 170.0 400.0 170.0 " style="vector-effect: non-scaling-stroke; stroke: grey; fill: none; opacity: 0.6; stroke-dasharray: 2; " transform="scale(1,-1)" /><polyline points="0.0 0.0 400.0 0.0 " style="vector-effect: non-scaling-stroke; stroke: black; fill: none; " transform="scale(1,-1)" /><polygon points="79.5 -4.0 80.5 -4.0 80.5 0.0 79.5 0.0 " style="vector-effect: non-scaling-stroke; " transform="scale(1,-1)" /><text x="80.0" style="font-family: Helvetica; text-anchor: middle; vector-effect: non-scaling-stroke; font-size: 7.0pt; " transform="translate(0 25.434895833333332) rotate(-0.0, 80.0, -12.717447916666666)" y="-12.717447916666666">under</text><polygon points="199.5 -4.0 200.5 -4.0 200.5 0.0 199.5 0.0 " style="vector-effect: non-scaling-stroke; " transform="scale(1,-1)" /><text x="200.0" style="font-family: Helvetica; text-anchor: middle; vector-effect: non-scaling-stroke; font-size: 7.0pt; " transform="translate(0 25.434895833333332) rotate(-0.0, 200.0, -12.717447916666666)" y="-12.717447916666666">grad</text><polygon points="319.5 -4.0 320.5 -4.0 320.5 0.0 319.5 0.0 " style="vector-effect: non-scaling-stroke; " transform="scale(1,-1)" /><text x="320.0" style="font-family: Helvetica; text-anchor: middle; vector-effect: non-scaling-stroke; font-size: 7.0pt; " transform="translate(0 22.044270833333332) rotate(-0.0, 320.0, -11.022135416666666)" y="-11.022135416666666">pos</text><text x="200.0" style="font-family: Helvetica; text-anchor: middle; vector-effect: non-scaling-stroke; font-size: 10.0pt; " transform="translate(0 68.75651041666666) rotate(-0.0, 200.0, -34.37825520833333)" y="-34.37825520833333">degree</text><polyline points="0.0 0.0 1.0409497792752502e-14 170.0 " style="vector-effect: non-scaling-stroke; stroke: black; fill: none; " transform="scale(1,-1)" /><polygon points="0.0 -0.5000000000000002 0.0 0.4999999999999998 -4.0 0.4999999999999998 -4.0 -0.5000000000000002 " style="vector-effect: non-scaling-stroke; " transform="scale(1,-1)" /><text x="-6.0" style="font-family: Helvetica; text-anchor: end; vector-effect: non-scaling-stroke; font-size: 7.0pt; " transform="translate(0 7.0) rotate(-0.0, -6.0, -3.5)" y="-3.5">0</text><polygon points="3.552713678800501e-15 56.166666666666664 3.552713678800501e-15 57.166666666666664 -3.9999999999999964 57.166666666666664 -3.9999999999999964 56.166666666666664 " style="vector-effect: non-scaling-stroke; " transform="scale(1,-1)" /><text x="-5

In [12]:
gplt = Vizagrams.flatten(ζ(plt)._2);
barpos = StructArray(map(x->(v=x.s.d[:id],xbar=x.geom.c[1],
            ybar=x.geom.c[2], hbar=x.geom.h, wbar =x.geom.w, color=x.s.d[:fill]),gplt))
sdata = DataFrame(barpos);

In [13]:
dv = leftjoin(dv,sdata,on=:v);

In [14]:
edges = DataFrame(:src=>[],:tgt=>[],:weight=>[])

for cols in [[:under,:grad],[:grad,:pos]]
    gdf = combine(groupby(df,cols),:count=>sum);
    gdf[!,:under_grad] = map(row->row[cols[1]]*"-"*row[cols[2]], eachrow(gdf))
    gdf[!,:src] = map(row->string(cols[1])*"_"*row[cols[1]], eachrow(gdf))
    gdf[!,:tgt] = map(row->string(cols[2])*"_"*row[cols[2]], eachrow(gdf))
    gdf[!,:weight] = gdf[!,:count_sum]
    gdf = gdf[!,[:src,:tgt,:weight]]
    edges = vcat(edges, gdf)
end
edges = leftjoin(edges,dv[!,[:xbar,:ybar,:hbar,:wbar,:v,:w,:country,:degree]],on=:src=>:v)
edges = rename(edges, :xbar=>:x_src,:ybar=>:y_src,:country=>:src_country,:degree=>:src_degree,:hbar=>:hbar_src)
edges = leftjoin(edges,dv[!,[:xbar,:ybar,:v,:country,:degree,:hbar]],on=:tgt=>:v)
edges = rename(edges, :xbar=>:x_tgt,:ybar=>:y_tgt,:country=>:tgt_country,:degree=>:tgt_degree,:hbar=>:hbar_tgt);
edges = dropmissing(edges);
edges[!,:h] = map(row->row[:hbar_src]*row[:weight]/row[:w], eachrow(edges));
temp = combine(groupby(edges,[:src_country,:src_degree]),:h=>cumsum,:src=>identity,:tgt=>identity)
temp = rename(temp, :src_identity=>:src,:tgt_identity=>:tgt,:h_cumsum=>:hc)
temp = temp[!,[:src,:tgt,:hc]]
edges = leftjoin(edges, temp, on=[:src,:tgt]);

gp = DataFrame()
for g in groupby(edges,[:src_country,:src_degree])
    g = sort(g,:hbar_src,rev=true)
    g[1,:y_src] = g[1,:y_src]+(g[1,:hbar_src]-g[1,:hc])/2
    # g[1,:y_tgt] = g[1,:y_tgt]+(g[1,:hbar_tgt]-g[1,:hc])/2
    for i in 2:size(g)[1]
        g[i,:y_src] = g[i,:y_src] + g[i,:hbar_src]/2 - g[i,:h]/2 - g[i-1,:hc]
        # g[i,:y_tgt] = g[i,:y_tgt] + g[i,:hbar_tgt]/2 - g[i,:h]/2 - g[i-1,:hc]
    end
    gp = vcat(gp,g)
end
edges = copy(gp)


temp = combine(groupby(sort(copy(edges),:hbar_src,rev=true),[:tgt_country,:tgt_degree]),:h=>cumsum,:src=>identity,:tgt=>identity)
temp = rename(temp, :src_identity=>:src,:tgt_identity=>:tgt,:h_cumsum=>:hc_tgt)
temp = temp[!,[:src,:tgt,:hc_tgt]]
edges = leftjoin(edges, temp, on=[:src,:tgt]);

In [15]:
gp = DataFrame()
for g in groupby(sort(copy(edges),:hbar_src,rev=true),[:tgt_country,:tgt_degree])
    g[1,:y_tgt] = g[1,:y_tgt] + (g[1,:hbar_tgt] - g[1,:h])/2
    for i in 2:size(g)[1]
        g[i,:y_tgt] = g[i,:y_tgt] + g[i,:hbar_tgt]/2 - g[i,:h]/2- g[i-1,:hc_tgt]
    end
    gp = vcat(gp,g)
end
gp = leftjoin(gp,dv[!,[:v,:color]],on=:src=>:v);

In [16]:
sankey = ∑(op=+) do row
    S(:strokeWidth=>0,:fill=>:grey,:fillOpacity=>0.2)SmoothWire(
        ptstart = [row[:x_src]+row[:wbar]/2,row[:y_src]],
        ptend = [row[:x_tgt]-row[:wbar]/2,row[:y_tgt]],
        hstart = row[:h],
        hend = row[:h],)
end

ed = StructArray(NamedTuple.(eachrow(gp)));
draw(sankey(ed)+plt.graphic(scaledata(plt)),height=500)

<svg height="500" viewBox="60.96666666666667 -154.86666666666667 278.06666666666666 156.4" width="896.7391304347825" preserveAspectRatio="xMidYMid meet" xmlns="http://www.w3.org/2000/svg" version="1.1"><path style="vector-effect: non-scaling-stroke; stroke: black; fill: grey; stroke-width: 0; fill-opacity: 0.2; " transform="scale(1,-1)" d="M 217.5 88.14000000000001 C 217.5 88.14000000000001 217.5 88.70666666666668 217.5 88.70666666666668 260.0 88.70666666666668 260.0 0.5666666666666662 302.5 0.5666666666666662 302.5 0.5666666666666662 302.5 1.6653345369377348e-16 302.5 1.6653345369377348e-16 C 260.0 1.6653345369377348e-16 260.0 88.14000000000001 217.5 88.14000000000001  Z" /><path style="vector-effect: non-scaling-stroke; stroke: black; fill: grey; stroke-width: 0; fill-opacity: 0.2; " transform="scale(1,-1)" d="M 217.5 0.34000000000000097 C 217.5 0.34000000000000097 217.5 3.286666666666668 217.5 3.286666666666668 260.0 3.286666666666668 260.0 3.5133333333333345 302.5 3.5133333333333345 302.5 3.5133333333333345 302.5 0.5666666666666675 302.5 0.5666666666666675 C 260.0 0.5666666666666675 260.0 0.34000000000000097 217.5 0.34000000000000097  Z" /><path style="vector-effect: non-scaling-stroke; stroke: black; fill: grey; stroke-width: 0; fill-opacity: 0.2; " transform="scale(1,-1)" d="M 217.5 69.07333333333334 C 217.5 69.07333333333334 217.5 70.88666666666667 217.5 70.88666666666667 260.0 70.88666666666667 260.0 5.326666666666669 302.5 5.326666666666669 302.5 5.326666666666669 302.5 3.5133333333333345 302.5 3.5133333333333345 C 260.0 3.5133333333333345 260.0 69.07333333333334 217.5 69.07333333333334  Z" /><path style="vector-effect: non-scaling-stroke; stroke: black; fill: grey; stroke-width: 0; fill-opacity: 0.2; " transform="scale(1,-1)" d="M 217.5 35.04666666666667 C 217.5 35.04666666666667 217.5 37.31333333333333 217.5 37.31333333333333 260.0 37.31333333333333 260.0 7.593333333333334 302.5 7.593333333333334 302.5 7.593333333333334 302.5 5.326666666666668 302.5 5.326666666666668 C 260.0 5.326666666666668 260.0 35.04666666666667 217.5 35.04666666666667  Z" /><path style="vector-effect: non-scaling-stroke; stroke: black; fill: grey; stroke-width: 0; fill-opacity: 0.2; " transform="scale(1,-1)" d="M 217.5 107.20666666666666 C 217.5 107.20666666666666 217.5 110.83333333333333 217.5 110.83333333333333 260.0 110.83333333333333 260.0 11.219999999999999 302.5 11.219999999999999 302.5 11.219999999999999 302.5 7.593333333333333 302.5 7.593333333333333 C 260.0 7.593333333333333 260.0 107.20666666666666 217.5 107.20666666666666  Z" /><path style="vector-effect: non-scaling-stroke; stroke: black; fill: grey; stroke-width: 0; fill-opacity: 0.2; " transform="scale(1,-1)" d="M 217.5 92.78666666666668 C 217.5 92.78666666666668 217.5 94.60000000000001 217.5 94.60000000000001 260.0 94.60000000000001 260.0 59.55333333333333 302.5 59.55333333333333 302.5 59.55333333333333 302.5 57.739999999999995 302.5 57.739999999999995 C 260.0 57.739999999999995 260.0 92.78666666666668 217.5 92.78666666666668  Z" /><path style="vector-effect: non-scaling-stroke; stroke: black; fill: grey; stroke-width: 0; fill-opacity: 0.2; " transform="scale(1,-1)" d="M 217.5 10.426666666666668 C 217.5 10.426666666666668 217.5 11.56 217.5 11.56 260.0 11.56 260.0 60.68666666666667 302.5 60.68666666666667 302.5 60.68666666666667 302.5 59.55333333333333 302.5 59.55333333333333 C 260.0 59.55333333333333 260.0 10.426666666666668 217.5 10.426666666666668  Z" /><path style="vector-effect: non-scaling-stroke; stroke: black; fill: grey; stroke-width: 0; fill-opacity: 0.2; " transform="scale(1,-1)" d="M 217.5 73.83333333333334 C 217.5 73.83333333333334 217.5 77.68666666666667 217.5 77.68666666666667 260.0 77.68666666666667 260.0 64.53999999999999 302.5 64.53999999999999 302.5 64.53999999999999 302.5 60.68666666666666 302.5 60.68666666666666 C 260.0 60.68666666666666 260.0 73.83333333333334 217.5 73.83333333333334  Z" /><path style="vector-effect: non-scaling-stroke; stroke: black; fill: g